In [ ]:
import pandas as pd
import numpy as np

uriage_copy = pd.read_csv("../csv/uriage.csv")

uriage = uriage_copy.copy()

uriage["purchase_date"] = pd.to_datetime(uriage["purchase_date"], format = "%Y-%m-%d %H:%M")

#대문자로 변환
uriage["item_name"] = uriage["item_name"].str.upper() #시리즈는 문자열이 아니기 때문에 .str을 붙여줘야 한다.

uriage['item_name'] = uriage['item_name'].str.replace(" ", "")
# case2 lambda
uriage['item_name'] = uriage['item_name'].apply(lambda x : x.replace(" ", ""))
# case3
def change(x):
    return x.replace(" ", "")
uriage['item_name'] = uriage['item_name'].apply(change)

#오름차순
uriage.sort_values(["item_name"], inplace=True)

#인덱스 재지정, 기존 인덱스 삭제
uriage.reset_index(drop=True, inplace=True)

#결측치확인
uriage.isna().sum()

#결측치 값에 해당하는 item_name들의 item_price 값 치환
flg_null = uriage['item_price'].isna()
uriage.loc[flg_null] # 전부 나옴
uriage.loc[flg_null, 'item_name'] #item name 나옴
uriage.loc[flg_null, 'item_name'].unique() # 중복데이터 삭제되고 전체 상품 리스트가 나옴
uriage.loc[(~flg_null)] #결측치가 아닌 데이터만 추출하겠다.
uriage.loc[(~flg_null) & (uriage['item_name'] == '상품A'), 'item_price'].max() # 결측치가 아닌 상품A의 item_price의 최대값
name_list = uriage.loc[flg_null, 'item_name'].unique()

for name in list(name_list): #리스트 개수만큼 반복
    price = uriage.loc[(~flg_null) & 
    (uriage['item_name'] == name), 'item_price'].mean() #상품 A일 때 한 번 바뀌고, B일 때 한 번 바뀌고... 
    #이 값을 결측치인 값에 집어넣어주면 된다.
    #결측치가 아닌 값을 찾기 위함
    
    uriage['item_price'].loc[(flg_null) & 
    (uriage['item_name'] == name)] = price #결측치이고, 조회한 값의 name이 같은 값을 price에 넣어주겠다.


# 값 치환 딕셔너리로 하는 법
price = uriage.groupby(['item_name']).mean()
uriage_dict = uriage.copy()
price_dict = price.to_dict()
price_dict = price_dict["item_price"]
uriage_dict['item_price'] = uriage_dict['item_price'].fillna(uriage_dict['item_name'].apply(lambda x: price_dict.get(x)))

# csv 파일을 로드에서 결합
1. csv 폴더에서 transaction_1, transaction_2, transaction_detail_1, transaction_detail_2 파일 4개 로드
2. transaction_1, transaction_2 데이터프레임을 단순한 행 결합
3. transaction_detail_1, transaction_detail_2 데이터프레임도 단순한 행 결합
4. 2번 과정에서 나온 데이터프레임과 3번 과정에서 나온 데이터프레임을 특정 조건을 갖고 열 결합
(특정 조건 찾아보기)  

In [ ]:
import pandas as pd
import numpy as np

tran1 = pd.read_csv('../csv/transaction_1.csv')
tran2 = pd.read_csv('../csv/transaction_2.csv')
tran_dt1 = pd.read_csv('../csv/transaction_detail_1.csv')
tran_dt2 = pd.read_csv('../csv/transaction_detail_2.csv')

# 데이터프레임 행 추가 단순 결합
tran = pd.concat([tran1, tran2], axis=0, ignore_index=True) # 옵션 기본 값이라 안적어도 됨
tran_dt = pd.concat([tran_dt1, tran_dt2])

tran_mer = pd.merge(tran, tran_dt, on = 'transaction_id', how='inner') # 데이터가 같아서 how 뭘 넣든 같은거 나올것.

tran_mer["payment_date"] = pd.to_datetime(tran_mer["payment_date"])

print(type(tran_mer["payment_date"]))

# 데이터 결합
1. customer_master, item_master 파일 로드
2. join_data 특정 조건에 맞게 열 추가
3. 결합이 된 데이터에서 quantity컬럼, item_price컬럼 존재.  
두 개의 곱을 해 새로운 파생변수 total_price 생성  
4. 6개의 csv 파일이 결합된 데이터프레임에 payment_date 컬럼의 값을 가지고 payment_month 라는 새로운 파생변수를 만든다. 파생변수에 들어갈 값은 년-월
5. payment_month를 기준으로 그룹화해서 total_price의 합을 구해라
6. 이를 시각화 (년월로 묶여서 얼마나 팔렸는지 알 수 있을 것)

In [ ]:
cs_master = pd.read_csv('../csv/customer_master.csv')
item_master = pd.read_csv('../csv/item_master.csv')

transaction1 = pd.merge(tran_mer, item_master, on='item_id', how='left')
transaction = pd.merge(transaction1, cs_master, on='customer_id', how='left')

transaction["total_price"] = transaction["quantity"] * transaction["item_price"]

transaction['payment_month'] = transaction['payment_date'].dt.strftime("%Y-%m")
                                                            #이 데이터 형식에서
                                                                #이거만 추출해줘잉

# payment_month 기준으로 그룹화해 total_price의 합을 추출 -> 시각화
group_data = transaction.groupby('payment_month').sum()['total_price']
            #이 데이터의          #인자값 ()              #컬럼 []

import matplotlib.pyplot as plt

plt.plot(group_data)
plt.show()

1. drinks.csv 파일 로드
2. 결측치 확인 
3. continent 컬럼에 결측치 존재 -> 결측치는 'OT'로 변경
4. continent별 spirit_serving 컬럼의 평균, 최소, 최대, 합을 출력
5. 이 데이터를 바형 그래프로 출력
6. total_liters_of_pure_alcohol 컬럼의 평균보다 높은 continent가 어디인지 확인

In [ ]:
import pandas as pd

drinks = pd.read_csv("../csv/drinks.csv")

drinks.isna().sum()

drinks['continent'] = drinks['continent'].fillna('OT')

drinks_group = drinks.groupby('continent').mean()['spirit_servings']

drinks_group.index

drinks_mean = drinks.groupby('continent').mean()['spirit_servings']
drinks_min = drinks.groupby('continent').min()['spirit_servings']
drinks_max = drinks.groupby('continent').max()['spirit_servings']
drinks_sum = drinks.groupby('continent').sum()['spirit_servings']


import matplotlib.pyplot as plt

a = drinks_group.index
a1 = drinks_mean.to_list()
a2 = drinks_min.to_list()
a3 = drinks_max.to_list()
a4 = drinks_sum.to_list()

plt.subplot(2, 2, 1)
plt.bar(a, a1)
plt.subplot(2, 2, 2)
plt.bar(a, a2)
plt.subplot(2, 2, 3)
plt.bar(a, a3)
plt.subplot(2, 2, 4)
plt.bar(a, a4)
plt.show()

# subplot(행, 열, 위치)
# subplot(2, 1, 2) 1
#                  2 #여기 나오는 것
# subplot(2, 2, 4) 1 2
#                  3 4 #여기 나옴

# 보충 : groupby 한 번에 묶기

In [ ]:
result = drinks.groupby('continent').agg(['mean', 'sum', 'min', 'max'])['spirit_servings']

mean = result['mean']
min = result['min']
max = result['max']
sum = result['sum']

x = result.index

# subplot(행의 수, 열의 수, 위치)
# 4개의 그래프 행의 수 * 열의 수 = 4
plt.subplot(2, 2, 1)
plt.bar(x, mean)
plt.subplot(2, 2, 2)
plt.bar(x, min)
plt.subplot(2, 2, 3)
plt.bar(x, max)
plt.subplot(2, 2, 4)
plt.bar(x, sum)
plt.show()

# 어느 대륙이 술 소비량이 가장 많은지 비교

In [ ]:
total_mean = drinks['total_litres_of_pure_alcohol'].mean()

mean_continent = drinks.groupby('continent').mean()['total_litres_of_pure_alcohol']

mean_continent[mean_continent >= total_mean] #조건식넣으면 True만 출력함
mean_continent >= total_mean


# 한 플롯에 다 그려보기
import numpy as np

cnt = len(x)

index = np.arange(cnt)

# 두 번 쓰면 그래프 하나에 두 개 나옴. 대신 겹치면 안됨 => 단, 약간의 편법임
bar_width = 0.1

plt.bar(index, means, bar_width, color='g', label='Mean')
plt.bar(index+bar_width, min, bar_width, color='b', label='Min') # x축의 값을 bar_width만큼 옆으로
plt.bar(index+bar_width*2, max, bar_width, color = 'r', label='Max')
plt.bar(index+bar_width*3, sum, bar_width, color='c', label='Sum')

plt.xticks(index, x) #x축의 뭔가의 설정을 변경하겠다~ index를 x값 이름으로 대체할래 
plt.legend() #범례

plt.show()

2017 연도별 데이터, 데이터 프레임 하나로 합치고 싶어 => 하나로 합쳐보기

In [ ]:
import pandas as pd
import os 

#지정된 경로에 있는 파일 목록을 불러오기 
files = os.listdir("../csv/2017") #파일명을 지정 #리스트 형태로 파일목록이 불러와진 것을 확인할 수 있음

# 파일 목록을 출력, 불러와서 concat() 함수 이용해 하나의 데이터프레임으로 결합
df_2017 = pd.DataFrame()

path = "../csv/2017/"

for file in files:
    df = pd.read_csv(path+file)
    df_2017 = pd.concat([df_2017, df], axis=0, ignore_index=True) #안적어도 되긴함

df_2017.info()


files18 = os.listdir("../csv/2018")

df_2018 = pd.DataFrame()

path18 = "../csv/2018/"

for file in files18:
    df18 = pd.read_csv(path18 + file)
    df_2018 = pd.concat([df_2018, df18])


total_df = pd.concat([df_2017, df_2018])

# json 파일 / xlsx 엑셀 파일 로드

In [ ]:
# jsno 형태는 dict 형태
# 이거는 csv와 동일
pd.read_json("../csv/2019/201901_expense_list.json")

# xlsx면 이렇게 해도 가능한데, xlsm이라 안됨
pd.read_excel("../csv/2020/202001_expense_list.xlsx")


!pip install openpyxl

pd.read_excel("../csv/2020/202001_expense_list.xlsx", engine="openpyxl")

# dataframe 정보확인
total_df.info()

# total_df
1. 컬럼의 이름 변경 ['nid', '제목', 'url', '부서레벨1', '부서레벨2', '부서레벨3', '부서레벨4', '부서레벨5', '집행연도', '집행일', '예산', '집행', '구분', '부서명', '집행일시', '집행장소', '집행목적', '대상인원', '결제방법', '집행금액']
2. 컬럼 삭제 -> 부서레벨3, 부서레벨4, 부서레벨5, 예산, 집행, 구분
3. 연도별 집행금액의 합 구하고 -> 시각화 (바형 그래프)
4. 연도, 월 별 집행금액 합계를 시각화 (바형 그래프)

In [ ]:
total_df = pd.concat([df_2017, df_2018])

total_df.columns = ['nid', '제목', 'url', '부서레벨1', '부서레벨2', '부서레벨3', '부서레벨4', '부서레벨5', '집행연도', '집행일', '예산', '집행', '구분', '부서명', '집행일시', '집행장소', '집행목적', '대상인원', '결제방법', '집행금액']

total_df.drop(['부서레벨3', '부서레벨4', '부서레벨5', '예산', '집행', '구분'], axis='columns', inplace=True)

# 집행 연도를 기준으로 그룹화해서 집행금액의 합계를 출력
month_total = total_df.groupby('집행연도').sum()['집행금액']


import matplotlib.pyplot as plt
import matplotlib

x = month_total.index # x축 -> index
y = month_total # y축 -> 집행금액의 합계

plt.bar(x, y)
plt.show()

In [ ]:
# 2 위 방법말고 다른 방법, 피벗 테이블 이용
# pivot_table()의 형태로 변경 시각화, groupby랑 같이 피벗테이블 형태로 묶는 것
month_total = pd.pivot_table(total_df, index=['집행연도'], values=['집행금액'], aggfunc='sum')

# 한글이 깨져서 한글 세팅
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

month_total.plot.bar()

month_total.plot.barh() #h만 붙이면 바 회전


# 연도별 월별
month_total = total_df.groupby(['집행연도', '집행일']).sum()['집행금액']

pivot_month = pd.pivot_table(total_df, index=['집행일'], columns=['집행연도'], values=['집행금액'], aggfunc='sum')

pivot_month.plot.bar()

# 집행연도와 부서레벨1 별 집행금액 평균이 어떠한가 그래프로 표시
lv1_mean = pd.pivot_table(total_df, index=['부서레벨1'], columns=['집행연도'], values=['집행금액'], aggfunc='mean')
